In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from multiprocessing import get_context
import shutil
from pathlib import Path
import numpy as np
import thor.data as data
import thor.parallel as parallel
import thor.visualize as visualize
import thor.analyze as analyze
import thor.option as option
import thor.grid as grid

notebook_name = "parallel_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parallel

In [4]:
 # Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T18:00"
end = "2005-11-13T20:00"

intervals = parallel.get_time_intervals(start, end)
output_parent = base_local / "runs/cpol_demo_parallel"
if output_parent.exists():
    shutil.rmtree(output_parent)
options_directory = output_parent / "options"

 # Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)

# Restrict the ERA5 data to a smaller region containing the CPOL radar
lon_range = [129, 133]
lat_range = [-14, -10]
era5_pl_options = data.era5.data_options(
    start=start, end=end, latitude_range=lat_range, longitude_range=lon_range
)
era5_sl_options = data.era5.data_options(
    start=start,
    end=end,
    data_format="single-levels",
    latitude_range=lat_range,
    longitude_range=lon_range,
)

data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

data.dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory)

altitude = list(np.arange(0, 20e3 + 500, 500))
altitude = [float(alt) for alt in altitude]
grid_options = grid.create_options(name="geographic", altitude=altitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)

# Create the track_options dictionary
track_options = option.default_track_options(dataset="cpol")
track_options.to_yaml(options_directory / "track.yml")
visualize_options = None

2024-10-18 14:01:28,715 - thor.utils - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/options/data.yml


2024-10-18 14:01:28,874 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-18 14:01:28,875 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-10-18 14:01:28,877 - thor.utils - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/options/grid.yml


In [5]:
with get_context("spawn").Pool(
    initializer=parallel.initialize_process(), processes=4
) as pool:
    results = []
    for i, time_interval in enumerate(intervals):
        args = [i, time_interval, data_options.copy(), grid_options.copy()]
        args += [track_options.model_copy(deep=True), visualize_options]
        args += [output_parent, "cpol"]
        args = tuple(args)
        results.append(pool.apply_async(parallel.track_interval, args))
    pool.close()
    pool.join()
    parallel.check_results(results)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



2024-10-18 14:01:36,465 - thor.utils - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_1/options/data.yml
2024-10-18 14:01:36,475 - thor.utils - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_1/options/grid.yml
2024-10-18 14:01:36,475 - thor.utils - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_0/options/data.yml
2024-10-18 14:01:36,486 - thor.utils - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_0/options/grid.yml
2024-10-18 14:01:36,525 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_1.
2024-10-18 14:01:36,525 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-18 14:01:36,530 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_0.
2024-10-18 14:01:36,530 - thor.track - INFO - Beginning simultaneous tracking

In [6]:
parallel.stitch_run(output_parent, intervals)

2024-10-18 14:02:06,952 - thor.parallel - INFO - Stitching all attribute, mask and record files.


2024-10-18 14:02:07,472 - thor.parallel - INFO - Stitching record files.
2024-10-18 14:02:07,493 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_parallel/records/filepaths/cpol.csv
2024-10-18 14:02:07,495 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_parallel/records/filepaths/cpol.yml
2024-10-18 14:02:07,497 - thor.parallel - INFO - Stitching attribute files.
2024-10-18 14:02:07,573 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_parallel/attributes/mcs/anvil/core.csv
2024-10-18 14:02:07,576 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_parallel/attributes/mcs/anvil/core.yml
2024-10-18 14:02:07,623 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_parallel/attributes/mcs/anvil/quality.csv
2024-10-18 14:02:07,626 - thor.wr

# Analyze

In [7]:
analysis_options = analyze.mcs.AnalysisOptions()
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent)

2024-10-18 14:02:10,736 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/velocities.csv
2024-10-18 14:02:10,739 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/velocities.yml
2024-10-18 14:02:10,953 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/quality.csv
2024-10-18 14:02:10,956 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/quality.yml
2024-10-18 14:02:11,026 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/classification.csv
2024-10-18 14:02:11,029 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/classification.yml


,,stratiform_offset,inflow,relative_stratiform_offset,tilt,propagation
time,universal_id,,,,,
2005-11-13 18:00:08,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 18:10:23,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:20:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:30:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:40:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:50:08,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 19:00:08,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 19:10:23,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 19:20:09,1,trailing,left,left,up-shear,shear-perpendicular


In [8]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="presentation"
)
start_time = np.datetime64("2005-11-13T18:00")
end_time = np.datetime64("2005-11-13T20:00")
visualize.attribute.mcs_series(
    output_parent, start_time, end_time, figure_options, parallel_figure=True
)

2024-10-18 14:02:12,293 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.180000.nc
2024-10-18 14:02:12,368 - thor.data.aura - INFO - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-10-18 14:02:14,828 - thor.visualize.attribute - DEBUG - Getting grid from dataset at time 2005-11-13T18:00:08.000000000.
2024-10-18 14:02:14,832 - thor.visualize.attribute - DEBUG - Rebuilding processed grid for time 2005-11-13T18:00:08.000000000.
2024-10-18 14:02:14,934 - thor.visualize.attribute - DEBUG - Creating grouped mask figure at time 2005-11-13T18:00:08.000000000.
2024-10-18 14:02:14,935 - thor.visualize.horizontal - INFO - No figsize provided. Using default.
2024-10-18 14:02:15,105 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-18 14:02:15,134 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-18 14:02:15,268 - thor.visualize.attribute - DEBUG - Saving mcs_velocity_analysis figure for 2005-11-13T18:00:08.000000000.



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-10-18 14:02:35,507 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.181000.nc
2024-10-18 14:02:36,323 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.182000.nc
2024-10-18 14:02:36,436 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.183000.nc
2024-10-18 14:02:36,530 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.184000.nc
2024-10-18 14:02:37,886 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.190000.nc
2024-10-18 14:02:38,125 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.185000.nc
2024-10-18 14:02:38,209 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.191000.nc
2024-10-18 14:02:38,624 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.192000.nc
2024-10-18 14:02:41,631 - thor.visualize.attribute - DEBUG - Getting grid from d